In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import polars as pl

In [2]:
from scripts import df_ct_combine

In [3]:
df_ct_combined = df_ct_combine.combine()

In [4]:
df_ct_combined

,customer_id,articles
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[505221004, 685687003, 685687004, 685687001, 5..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[688873012, 501323011, 598859003, 688873020, 6..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[531310002, 529841001, 662980002, 496111003, 5..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[501820043, 501820043, 674681001, 671505001, 6..."
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,"[756322001, 822311012, 891763003, 889133001]"
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,"[903647001, 903647001]"
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,"[913597001, 903420001, 761406001]"
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,"[673186001, 717205001, 669713004, 514134001, 5..."


In [5]:
df_a = pd.read_csv("~/CIS_4496_Project/data/raw/articles.csv")
df_c = pd.read_csv("~/CIS_4496_Project/data/raw/customers.csv")
df_t = pd.read_csv("~/CIS_4496_Project/data/raw/transactions_train.csv")

In [6]:
df_a

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [7]:
df_a_sub = df_a[['article_id','prod_name','product_type_name','product_group_name','graphical_appearance_name','colour_group_name'
                         ,'perceived_colour_value_name','perceived_colour_master_name','department_name','index_name','index_group_name'
                         ,'section_name','garment_group_name','detail_desc']]

for i in df_a_sub.columns[1:]:
    df_a_sub[i] = df_a_sub[i].str.replace(" ","")
    
article_cols = ['prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name', 'detail_desc']
df_a_sub['combined_info'] = df_a_sub[article_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

df_a_final = df_a_sub[['article_id','combined_info']]

/tmp/ipykernel_19793/3920140751.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_sub[i] = df_a_sub[i].str.replace(" ","")
/tmp/ipykernel_19793/3920140751.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_sub['combined_info'] = df_a_sub[article_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [8]:
df_a_final

,article_id,combined_info
0,108775015,Straptop Vesttop GarmentUpperbody Solid Black ...
1,108775044,Straptop Vesttop GarmentUpperbody Solid White ...
2,108775051,Straptop(1) Vesttop GarmentUpperbody Stripe Of...
3,110065001,OPT-shirt(Idro) Bra Underwear Solid Black Dark...
4,110065002,OPT-shirt(Idro) Bra Underwear Solid White Ligh...
...,...,...
105537,953450001,5pkregularPlacement1 Socks Socks&Tights Placem...
105538,953763001,SPORTMalagatank Vesttop GarmentUpperbody Solid...
105539,956217002,Cartwheeldress Dress GarmentFullbody Solid Bla...
105540,957375001,CLAIREHAIRCLAW Hairclip Accessories Solid Blac...


In [9]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_a_final['combined_info'].values.astype('U'))

In [10]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:
results = {}
item_results = {}

for idx, row in df_a.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df_a['article_id'][i]) for i in similar_indices]
    items_only = [df_a['article_id'][i] for i in similar_indices]
    
    results[row['article_id']] = similar_items[1:]
    item_results[row['article_id']] = items_only[1:]
    
print('done!')

done!


In [12]:
def item(id):
    return df_a.loc[df_a['article_id'] == id]['detail_desc'].tolist()[0].split(' - ')[0]

In [13]:
def recommend(item_id, num):
    #print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print(f'Recommending {num} products similar to {item_id}.')
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print(f"{rec[1]}")

In [14]:
#recs = results[item_id][:num]
output = item_results[663713001][:12]
print(output)

[487722001, 487722007, 487722008, 487722005, 650037001, 667641001, 529008003, 639525003, 614496001, 638416001, 635530001, 526823001]


In [57]:
predictions = []
for row in df_ct_combined['articles']:
    length = len(row)
    if length <= 12 and 12 % length == 0:
        pred1 = []
        for i in range(length):
            nextProd = int(row[i])
            numRecs = int(12 / length)
            pred1.append(item_results[nextProd][:numRecs])
        predictions.append(pred1)
    elif length <= 12 and 12 % length != 0 and length != 5:
        pred2 = []
        for i in range(length):
            nextProd = int(row[i])
            pred2.append(item_results[nextProd][:1])
        leftover = 12 - length
        for i in range(leftover):
            leftoverProd = int(row[i])
            pred2.append(item_results[leftoverProd][1:2])
        predictions.append(pred2)
    elif length == 5:
        pred3 = []
        for i in range(5):
            nextProd = int(row[i])
            pred3.append(item_results[nextProd][:1])
        #for i in range(7):
            #leftoverProd = int(row[i])
            #predictions.append(item_results[leftoverProd][1:2]) 
        predictions.append(pred3)
    elif length > 12:
        pred4 = []
        for i in range(12):
            nextProd = int(row[i])
            pred4.append(item_results[nextProd][:1])
        predictions.append(pred4)

In [58]:
print(len(predictions))

1362281


In [59]:
submission = pd.DataFrame(columns=['customer_id', 'prediction'])
submission['customer_id'] = df_ct_combined['customer_id']
submission['prediction'] = predictions
submission

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[[487722001], [541518001], [487722001], [57802..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[[505221001], [685687002], [685687002], [68568..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[[688873012], [501323012], [598859004], [69563..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[[531310001], [529841002], [662980001], [49611..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[[501820003], [501820003], [674681002], [67150..."
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,"[[756322009, 756322004, 756322006], [822311001..."
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,"[[903647002, 772907001, 827482001, 673677003, ..."
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,"[[913597002, 640131002, 640131006, 640131004],..."
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,"[[483011005], [664074013], [669713001], [51413..."


In [60]:
def convert(predStr):
    return predStr.replace('[', '').replace(']', '').replace(',', '')

submission['prediction'] = submission['prediction'].apply(str).apply(convert)
submission

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,487722001 541518001 487722001 578020004 723529...
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001 685687002 685687002 685687002 505221...
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012 501323012 598859004 695632015 688873...
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,531310001 529841002 662980001 496111004 561752...
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820003 501820003 674681002 671505002 671505...
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,756322009 756322004 756322006 822311001 822311...
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,903647002 772907001 827482001 673677003 673677...
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,913597002 640131002 640131006 640131004 594264...
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,483011005 664074013 669713001 514134005 581781...


In [61]:
customers_without_transactions = pd.DataFrame(columns=['customer_id', 'prediction'])
transaction_customers = np.unique(df_t['customer_id'])
all_customers = df_c['customer_id']
customers_without_transactions['customer_id'] = list(set(transaction_customers).symmetric_difference(set(all_customers)))

In [62]:
customers_without_transactions

,customer_id,prediction
0,5c85b54f377c6d2cf32ec85f46fdd7f808144dfaa1f37c...,NaN
1,959e5bee8100f41b7d9234c18673725b4a1118779ea757...,NaN
2,54a276740b87181718453fda59c12cb39c8a16fecf8b74...,NaN
3,2664be86093e6214124818c65956d0e5c0b8e281f889e2...,NaN
4,bef8e685a352cc8fe441c6af37edd71ac435910fe3dee4...,NaN
...,...,...
9694,2974ea0a35284e307edc10b0059fb46b471eeb6dbc63ca...,NaN
9695,0b994e92fc88f6802d893f1d025a2c97573f266e51d929...,NaN
9696,b8630d86844399bb5e407e10e9b2ff80c964cd4fdd4edd...,NaN
9697,7b591d201b69e454d5b285cde74d79a5f7a69a031a012f...,NaN


In [63]:
df_t['article_id'].value_counts().nlargest(12)


706016001    50287
706016002    35043
372860001    31718
610776002    30199
759871002    26329
464297007    25025
372860002    24458
610776001    22451
399223001    22236
706016003    21241
720125001    21063
156231001    21013
Name: article_id, dtype: int64

In [64]:
popular = df_t['article_id'].value_counts()[:12].index.tolist()
popular

/tmp/ipykernel_19793/3723116339.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  popular = df_t['article_id'].value_counts()[:12].index.tolist()


[706016001,
 706016002,
 372860001,
 610776002,
 759871002,
 464297007,
 372860002,
 610776001,
 399223001,
 706016003,
 720125001,
 156231001]

In [65]:
most_popular = []
for row in customers_without_transactions['customer_id']:
    most_popular.append(popular) 


In [66]:
customers_without_transactions['prediction'] = most_popular
customers_without_transactions

,customer_id,prediction
0,5c85b54f377c6d2cf32ec85f46fdd7f808144dfaa1f37c...,"[706016001, 706016002, 372860001, 610776002, 7..."
1,959e5bee8100f41b7d9234c18673725b4a1118779ea757...,"[706016001, 706016002, 372860001, 610776002, 7..."
2,54a276740b87181718453fda59c12cb39c8a16fecf8b74...,"[706016001, 706016002, 372860001, 610776002, 7..."
3,2664be86093e6214124818c65956d0e5c0b8e281f889e2...,"[706016001, 706016002, 372860001, 610776002, 7..."
4,bef8e685a352cc8fe441c6af37edd71ac435910fe3dee4...,"[706016001, 706016002, 372860001, 610776002, 7..."
...,...,...
9694,2974ea0a35284e307edc10b0059fb46b471eeb6dbc63ca...,"[706016001, 706016002, 372860001, 610776002, 7..."
9695,0b994e92fc88f6802d893f1d025a2c97573f266e51d929...,"[706016001, 706016002, 372860001, 610776002, 7..."
9696,b8630d86844399bb5e407e10e9b2ff80c964cd4fdd4edd...,"[706016001, 706016002, 372860001, 610776002, 7..."
9697,7b591d201b69e454d5b285cde74d79a5f7a69a031a012f...,"[706016001, 706016002, 372860001, 610776002, 7..."


In [67]:
customers_without_transactions['prediction'] = customers_without_transactions['prediction'].apply(str).apply(convert)
customers_without_transactions

,customer_id,prediction
0,5c85b54f377c6d2cf32ec85f46fdd7f808144dfaa1f37c...,706016001 706016002 372860001 610776002 759871...
1,959e5bee8100f41b7d9234c18673725b4a1118779ea757...,706016001 706016002 372860001 610776002 759871...
2,54a276740b87181718453fda59c12cb39c8a16fecf8b74...,706016001 706016002 372860001 610776002 759871...
3,2664be86093e6214124818c65956d0e5c0b8e281f889e2...,706016001 706016002 372860001 610776002 759871...
4,bef8e685a352cc8fe441c6af37edd71ac435910fe3dee4...,706016001 706016002 372860001 610776002 759871...
...,...,...
9694,2974ea0a35284e307edc10b0059fb46b471eeb6dbc63ca...,706016001 706016002 372860001 610776002 759871...
9695,0b994e92fc88f6802d893f1d025a2c97573f266e51d929...,706016001 706016002 372860001 610776002 759871...
9696,b8630d86844399bb5e407e10e9b2ff80c964cd4fdd4edd...,706016001 706016002 372860001 610776002 759871...
9697,7b591d201b69e454d5b285cde74d79a5f7a69a031a012f...,706016001 706016002 372860001 610776002 759871...


In [68]:
df_submission_all = submission.append(customers_without_transactions, ignore_index=True)

/tmp/ipykernel_19793/2791921899.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_submission_all = submission.append(customers_without_transactions, ignore_index=True)


In [69]:
df_submission_all

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,487722001 541518001 487722001 578020004 723529...
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001 685687002 685687002 685687002 505221...
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012 501323012 598859004 695632015 688873...
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,531310001 529841002 662980001 496111004 561752...
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820003 501820003 674681002 671505002 671505...
...,...,...
1371975,2974ea0a35284e307edc10b0059fb46b471eeb6dbc63ca...,706016001 706016002 372860001 610776002 759871...
1371976,0b994e92fc88f6802d893f1d025a2c97573f266e51d929...,706016001 706016002 372860001 610776002 759871...
1371977,b8630d86844399bb5e407e10e9b2ff80c964cd4fdd4edd...,706016001 706016002 372860001 610776002 759871...
1371978,7b591d201b69e454d5b285cde74d79a5f7a69a031a012f...,706016001 706016002 372860001 610776002 759871...


In [70]:
df_submission_all.to_csv('submission3.csv', index = False)